In [1]:
import pystac_client
import odc.stac

In [2]:
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

In [3]:
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

In [ ]:
# Set a bounding box
# [xmin, ymin, xmax, ymax] in latitude and longitude
bbox = [153.3, -27.5, 153.4, -27.6]

# Set a start and end date
start_date = "1987-01-01"
end_date = "1987-12-31"

# Set product ID as the STAC "collection"
collections = ["ga_ls_fc_pc_cyear_3"]

In [5]:
# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")

items

Found: 2 datasets


[<Item id=41d9b9e6-c168-436a-b639-f2446f508d6e>,
 <Item id=b3c3be4c-eb3b-47f4-a130-4bd00f031bc4>]

In [6]:
ds = odc.stac.load(
             items=items,
             crs="EPSG:3577",
             lat=(bbox[1], bbox[3]),
             lon=(bbox[0], bbox[2]),
             time=("2015-01-01", "2018-12-31"))

ds


<xarray.Dataset> Size: 7MB
Dimensions:      (y: 424, x: 384, time: 1)
Coordinates:
  * y            (y) float64 3kB -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 3kB 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 4B 3577
  * time         (time) datetime64[ns] 8B 1987-01-01
Data variables:
    qa           (time, y, x) float32 651kB 0.0 0.0 0.0 0.0 ... 2.0 2.0 2.0 2.0
    bs_pc_10     (time, y, x) float32 651kB nan nan nan nan ... 10.0 9.0 8.0 8.0
    bs_pc_50     (time, y, x) float32 651kB nan nan nan nan ... 9.0 9.0 14.0
    bs_pc_90     (time, y, x) float32 651kB nan nan nan nan ... 20.0 15.0 17.0
    pv_pc_10     (time, y, x) float32 651kB nan nan nan nan ... 53.0 53.0 50.0
    pv_pc_50     (time, y, x) float32 651kB nan nan nan nan ... 58.0 55.0 55.0
    pv_pc_90     (time, y, x) float32 651kB nan nan nan nan ... 70.0 66.0 58.0
    npv_pc_10    (time, y, x) float32 651kB nan nan nan nan ... 20.0 24.0 27.0
    npv_pc_50    (time, y, x) float32 651kB nan nan nan nan ... 26.0 31.0 32.0
    npv_pc_90    (time, y, x) float32 651kB nan nan nan nan ... 32.0 35.0 35.0

In [7]:
# Print the loaded dataset
print("Dataset info:")
print(ds)

# Print the type of the bs_pc_10 data
print("\nType of bs_pc_10:")
print(type(ds.bs_pc_10))

# Print the data type (dtype) of the band values
print("\nData type of bs_pc_10 values:")
print(ds.bs_pc_10.dtype)

Dataset info:
<xarray.Dataset> Size: 7MB
Dimensions:      (y: 424, x: 384, time: 1)
Coordinates:
  * y            (y) float64 3kB -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 3kB 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 4B 3577
  * time         (time) datetime64[ns] 8B 1987-01-01
Data variables:
    qa           (time, y, x) float32 651kB 0.0 0.0 0.0 0.0 ... 2.0 2.0 2.0 2.0
    bs_pc_10     (time, y, x) float32 651kB nan nan nan nan ... 10.0 9.0 8.0 8.0
    bs_pc_50     (time, y, x) float32 651kB nan nan nan nan ... 9.0 9.0 14.0
    bs_pc_90     (time, y, x) float32 651kB nan nan nan nan ... 20.0 15.0 17.0
    pv_pc_10     (time, y, x) float32 651kB nan nan nan nan ... 53.0 53.0 50.0
    pv_pc_50     (time, y, x) float32 651kB nan nan nan nan ... 58.0 55.0 55.0
    pv_pc_90     (time, y, x) float32 651kB nan nan nan nan ... 70.0 66.0 58.0
    npv_pc_10    (time, y, x) float32 651kB nan nan nan nan ... 20.0 24.0 27.0
    npv_pc_5

In [8]:
ds.bs_pc_10[0]

<xarray.DataArray 'bs_pc_10' (y: 424, x: 384)> Size: 651kB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [14., 18., 16., ..., 10.,  3.,  8.],
       [16., 15., 21., ..., 11.,  4.,  9.],
       [22., 19., 24., ...,  9.,  8.,  8.]], dtype=float32)
Coordinates:
  * y            (y) float64 3kB -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 3kB 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 4B 3577
    time         datetime64[ns] 8B 1987-01-01

In [9]:
ds.bs_pc_10[1]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
ds.bs_pc_10[0][0]

In [ ]:
bbox = [146.29, -39.10, 146.39, -39.00]

# Set a start and end date
start_date = "2015-01-01"
end_date = "2020-12-31"

# Set product ID as the STAC "collection"
collections = ["ga_ls_fc_pc_cyear_3"]

# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")


ds = odc.stac.load(
             items=items,
             crs="EPSG:3577",
             lat=(bbox[1], bbox[3]),
             lon=(bbox[0], bbox[2]),
             time=("2015-01-01", "2018-12-31"))

ds

In [ ]:
ds.bs_pc_10.__len__()

In [ ]:
ds.bs_pc_10[5]

In [ ]:
ds.bs_pc_10[5][5]

In [ ]:
ds.pv_pc_50[5][5]

In [ ]:
ds.pv_pc_50[5].mean().values

In [ ]:
for item in ds:

    if item == 'qa':
        continue

    mean = ds[item].mean().values

    print(f"Mean value of {item}: {mean}")